# RareSkills Zero Knowledge Week 

L\mathbf{\vec{[s]_1}}\odot R\mathbf{\vec{[s]_2}} = O\mathbf{\vec{[s]}_{1}}\odot\vec{[G_2]_2}


In [22]:
from numpy import poly1d
from py_ecc.bn128 import G1, G2, pairing, add, multiply, eq, neg, G12
import numpy as np
from scipy.interpolate import lagrange
import galois
from py_ecc.fields.field_properties import (
    field_properties,
)
from functools import reduce

field_modulus = field_properties["bn128"]["field_modulus"]
print(f"field_modulus: {field_modulus}")

GF = galois.GF(field_modulus)
# out = x⁴ - 5y²x²
# 1, out, x, y, v1, v2, v3
L = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, field_modulus-5, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

R = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
])

O = np.array([
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, field_modulus-1, 0],
])

L_galois = GF(L)
R_galois = GF(R)
O_galois = GF(O)

x = GF(4)
y = GF(field_modulus-2)
v1 = x * x
v2 = v1 * v1         # x^4
v3 = GF(field_modulus-5)*y * y
out = v3*v1 + v2    # -5y^2 * x^2

witness = GF(np.array([1, out, x, y, v1, v2, v3]))

assert all(np.equal(np.matmul(L_galois, witness) * np.matmul(R_galois, witness), np.matmul(O_galois, witness))), "not equal"

def interpolate_column(col):
    xs = GF(np.array([1,2,3,4]))
    return galois.lagrange_poly(xs, col)

# axis 0 is the columns. apply_along_axis is the same as doing a for loop over the columns and collecting the results in an array
U_polys = np.apply_along_axis(interpolate_column, 0, L_galois)
V_polys = np.apply_along_axis(interpolate_column, 0, R_galois)
W_polys = np.apply_along_axis(interpolate_column, 0, O_galois)

#print(U_polys[:2])
#print(V_polys[:2])
#print(W_polys[:1])

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

term_1 = inner_product_polynomials_with_witness(U_polys, witness)
term_2 = inner_product_polynomials_with_witness(V_polys, witness)
term_3 = inner_product_polynomials_with_witness(W_polys, witness)

# t = (x - 1)(x - 2)(x - 3)(x - 4)
t = galois.Poly([1, field_modulus-1], field = GF) * galois.Poly([1, field_modulus-2], field = GF) * galois.Poly([1, field_modulus-3], field = GF) * galois.Poly([1, field_modulus-4], field = GF)

h = (term_1 * term_2 - term_3) // t

assert term_1 * term_2 == term_3 + h * t, "division has a remainder"

print(term_1)
print(term_2)
print(term_3)
print(h)
print(t)

# res = multiply(G1, term_1(2)) * multiply(G2m term_2(2)) == multiply(G1, term_3(2)) + multiply(G2, h(2) * t(2))
print(term_1(2))
print(type(term_1(2)))
#A = multiply(G1, term_1(2))
#B = multiply(G2, term_2(2))
#C = multiply(G1, term_3(2))
#D = multiply(G2, h(2) * t(2))
#print(pairing(B,A))
#print(pairing(D, C))


field_modulus: 21888242871839275222246405745257275088696311157297823662689037894645226208583
21888242871839275222246405745257275088696311157297823662689037894645226208582x^3 + 21888242871839275222246405745257275088696311157297823662689037894645226208580x^2 + 28x + 21888242871839275222246405745257275088696311157297823662689037894645226208563
11x^3 + 21888242871839275222246405745257275088696311157297823662689037894645226208502x^2 + 178x + 21888242871839275222246405745257275088696311157297823662689037894645226208479
82x^3 + 21888242871839275222246405745257275088696311157297823662689037894645226207833x^2 + 1916x + 21888242871839275222246405745257275088696311157297823662689037894645226207351
21888242871839275222246405745257275088696311157297823662689037894645226208572x^2 + 21888242871839275222246405745257275088696311157297823662689037894645226208521x + 138
x^4 + 21888242871839275222246405745257275088696311157297823662689037894645226208573x^3 + 35x^2 + 218882428718392752222464057452572750886

In [66]:
import random
# Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = 1#random.randint(1,1000)
y = 2#random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])
result = C.dot(w) == np.multiply(A.dot(w),B.dot(w))
assert result.all(), "result contains an inequality"

def interpolate_column(col):
    xs = np.array([1,2,3])
    z = lagrange(xs, col)
    #print(z)
    return z

# axis 0 is the columns. apply_along_axis is the same as doing a for loop over the columns and collecting the results in an array
A_d = A.dot(w);
B_d = B.dot(w);
C_d = C.dot(w);

U_polys = [interpolate_column(A[:, i]) for i in range(A.shape[1])]
V_polys = [interpolate_column(B[:, i]) for i in range(B.shape[1])]
W_polys = [interpolate_column(C[:, i]) for i in range(C.shape[1])]

print(f"witness: {w}")
def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

def multiply_and_sum(arr1, arr2):
    return sum(x * y for x, y in zip(arr1, arr2))

term_1 = multiply_and_sum(U_polys, w)
term_2 = multiply_and_sum(V_polys, w)
term_3 = multiply_and_sum(W_polys, w)

t = poly1d([1, -1])*poly1d([1, -2])*poly1d([1, -3])
result_poly = term_1 * term_2 - term_3
# Divide result_poly by t using numpy.polydiv, which returns the quotient and remainder
quotient, remainder = np.polydiv(result_poly, t)
quotient_poly = poly1d(quotient)
remainder_poly = poly1d(remainder)
print(term_1 * term_2)
print(term_3 + quotient_poly * t + remainder_poly)
assert term_1 * term_2 == term_3 + quotient_poly * t + remainder_poly, "result contains an inequality"

witness: [ 1 14  1  2  3  6]
      4      3      2
-3.5 x + 20 x - 32 x + 11.5 x + 7
      4      3      2
-3.5 x + 20 x - 32 x + 11.5 x + 7
